In [17]:
# ==============================================================================
# CÉLULA 1: IMPORTAÇÕES (CORRIGIDO)
# ==============================================================================
# Bibliotecas padrão para manipulação de arquivos e sistema operacional
import os
import re
from pathlib import Path

# Biblioteca principal para manipulação e análise de dados
import pandas as pd

# Usado para escrever em arquivos Excel com múltiplas abas
from openpyxl import Workbook

# A linha abaixo foi removida pois 'dataframe_to_sheet' não é mais usada
# from openpyxl.utils.dataframe import dataframe_to_sheet

print("Bibliotecas importadas com sucesso.")
print(f"Pandas versão: {pd.__version__}")

Bibliotecas importadas com sucesso.
Pandas versão: 2.3.1


In [18]:
# ==============================================================================
# CÉLULA 2: CONFIGURAÇÃO DE DIRETÓRIOS
# ==============================================================================
try:
    # Em um ambiente de script .py, __file__ existe
    NB_DIR = Path(__file__).parent.resolve()
except NameError:
    # Em um notebook Jupyter, usamos Path() para obter o diretório atual
    NB_DIR = Path().resolve()

# Assume que a estrutura é /repo/notebooks/ e os dados estão em /repo/data
REPO_ROOT = NB_DIR.parent
DATA_DIR = (REPO_ROOT / "data").resolve()
OUT_DIR = (REPO_ROOT / "outputs").resolve()

# Cria os diretórios se eles não existirem
DATA_DIR.mkdir(exist_ok=True)
OUT_DIR.mkdir(exist_ok=True)

print(f"Pasta de DADOS (Entrada): {DATA_DIR}")
print(f"Pasta de RELATÓRIOS (Saída): {OUT_DIR}")

Pasta de DADOS (Entrada): C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\data
Pasta de RELATÓRIOS (Saída): C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs


In [ ]:
# ==============================================================================
# CÉLULA 3: FUNÇÃO PARA CONSOLIDAR DADOS-FONTE (VERSÃO CORRIGIDA E FINAL)
# ==============================================================================
def consolidar_dados_mensais(
    mes_ano_base: str, pasta_dados: Path
) -> pd.DataFrame | None:
    """
    Lê o arquivo base 'vigiram' e o enriquece com a coluna 'Material Análise' do arquivo 'its',
    que é a fonte correta para o tipo de amostra.

    Args:
        mes_ano_base (str): A base do nome do arquivo (ex: "jan25").
        pasta_dados (Path): O caminho para o diretório 'data'.

    Returns:
        pd.DataFrame ou None: Um DataFrame consolidado ou None se os arquivos
                              principais não forem encontrados.
    """
    print(f"🔄 Processando base: {mes_ano_base}...")
    try:
        # 1. Carregar os arquivos de entrada
        path_base = pasta_dados / f"vigiram-{mes_ano_base}.csv"
        path_its = pasta_dados / f"its-{mes_ano_base}.csv"

        df_base = pd.read_csv(path_base, dtype=str, low_memory=False)
        df_its = pd.read_csv(path_its, dtype=str, low_memory=False)

        # 2. Preparar o DataFrame 'its' para o merge
        # Renomeia a coluna chave para 'solicitacao' e a coluna de interesse para um nome único
        df_its_renamed = df_its.rename(
            columns={
                "Solicitação": "solicitacao",
                "Material Análise": "fonte_correta_amostra",
            }
        )

        # Seleciona apenas a chave e a coluna de interesse para evitar duplicatas e conflitos
        df_its_subset = df_its_renamed[["solicitacao", "fonte_correta_amostra"]]

        # 3. Executar o merge
        # Usamos um 'left merge' para garantir que todos os registros do arquivo base sejam mantidos.
        # As informações de amostra do 'its' serão adicionadas onde houver correspondência de 'solicitacao'.
        df_final = pd.merge(df_base, df_its_subset, on="solicitacao", how="left")

        print(
            f"✅ Dados de '{mes_ano_base}' consolidados. Fonte de amostra ('Material Análise' do ITS) vinculada."
        )
        return df_final

    except FileNotFoundError as e:
        print(
            f"⚠️ Arquivo principal faltando para '{mes_ano_base}': {e.filename}. Pulando este mês."
        )
        return None
    except Exception as e:
        print(f"❌ Erro inesperado ao processar '{mes_ano_base}': {e}")
        return None

In [ ]:
# ==============================================================================
# CÉLULA 4: FUNÇÃO PARA LIMPEZA E ENRIQUECIMENTO DOS DADOS (VERSÃO CORRIGIDA E FINAL)
# ==============================================================================
def limpar_e_preparar_dados(df: pd.DataFrame) -> pd.DataFrame:
    """
    Limpa, formata e enriquece o DataFrame consolidado para análise da CCIRAS.

    Args:
        df (pd.DataFrame): O DataFrame bruto consolidado.

    Returns:
        pd.DataFrame: O DataFrame pronto para a geração dos relatórios.
    """
    # Renomear colunas para nomes mais intuitivos e remover espaços
    mapeamento_colunas = {
        # **AQUI ESTÁ A CORREÇÃO PRINCIPAL**: Mapeamos a coluna que foi
        # corretamente extraída do ITS na etapa de consolidação.
        "fonte_correta_amostra": "tipo_amostra",
        "data_area_executora": "data_liberacao",
        "paciente": "nome_paciente",
        "solicitacao": "prontuario",
        "unidade_solicitante": "unidade_solicitante",
        "Antibiótico": "antibiotico_testado",
        "RSI": "resultado_rsi",
        "Mic": "valor_mic",
        "Perfil de Resistência": "perfil_resistencia",
        "origem": "origem_atendimento",
        "Tipo Alta": "tipo_alta",
    }
    # Renomeia apenas as colunas que existem no DataFrame
    df = df.rename(
        columns={k: v for k, v in mapeamento_colunas.items() if k in df.columns}
    )

    # Padronização de texto (remove espaços e converte para maiúsculas)
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].str.strip().str.upper()

    # Conversão de datas
    df["data_liberacao"] = pd.to_datetime(df["data_liberacao"], errors="coerce")

    # Preencher valores nulos em colunas críticas para evitar erros na análise
    colunas_para_preencher = [
        "microorganismo",
        "antibiotico_testado",
        "resultado_rsi",
        "unidade_solicitante",
        "tipo_amostra",
    ]
    for col in colunas_para_preencher:
        if col in df.columns:
            df[col] = df[col].fillna("N/A")
        else:
            # Se uma coluna crítica ainda não existir, cria com N/A para evitar erros futuros
            df[col] = "N/A"

    # Criar coluna de origem simplificada (Hospitalar vs. Ambulatorial)
    df["origem_atendimento_simplificada"] = df["origem_atendimento"].apply(
        lambda x: (
            "AMBULATORIAL"
            if isinstance(x, str) and x.startswith("AMBUL")
            else "HOSPITALAR"
        )
    )

    return df

In [21]:
# ==============================================================================
# CÉLULA 5: FUNÇÕES PARA GERAÇÃO DAS ABAS ANALÍTICAS (VERSÃO CORRIGIDA)
# ==============================================================================


def gerar_aba_dashboard(df: pd.DataFrame, df_contagens: pd.DataFrame) -> pd.DataFrame:
    """Gera a aba de Dashboard com indicadores chave."""
    total_pac_positivos = df_contagens.get("positivos", 0)
    total_pac_negativos = df_contagens.get("negativos", 0)

    total_amostras = df["amostra"].nunique()
    total_isolados = df[df["microorganismo"] != "N/A"]["microorganismo"].count()

    isolados_acineto = df[df["microorganismo"] == "ACINETOBACTER BAUMANNII"].shape[0]
    isolados_klebsiella = df[df["microorganismo"] == "KLEBSIELLA PNEUMONIAE"].shape[0]
    isolados_pseudo = df[df["microorganismo"] == "PSEUDOMONAS AERUGINOSA"].shape[0]
    isolados_staph = df[df["microorganismo"] == "STAPHYLOCOCCUS AUREUS"].shape[0]
    isolados_entero = df[
        df["microorganismo"].str.contains("ENTEROCOCCUS", na=False)
    ].shape[0]

    def calcular_taxa_resistencia(mic, antib):
        testes = df[
            (df["microorganismo"] == mic) & (df["antibiotico_testado"] == antib)
        ]
        if testes.empty:
            return 0.0
        resistentes = testes[testes["resultado_rsi"] == "R"].shape[0]
        return (resistentes / len(testes) * 100) if len(testes) > 0 else 0.0

    taxa_acineto_carba = calcular_taxa_resistencia(
        "ACINETOBACTER BAUMANNII", "MEROPENEM"
    )
    taxa_mrsa = calcular_taxa_resistencia("STAPHYLOCOCCUS AUREUS", "OXACILINA")
    taxa_vre = calcular_taxa_resistencia("ENTEROCOCCUS FAECALIS", "VANCOMICINA")

    data = {
        "Indicador": [
            "Total de Pacientes com Cultura Positiva",
            "Total de Pacientes com Cultura Negativa",
            "Total de Amostras Coletadas",
            "Total de Microrganismos Isolados",
            "Nº de Isolados de Acinetobacter baumannii",
            "Nº de Isolados de Klebsiella pneumoniae",
            "Nº de Isolados de Pseudomonas aeruginosa",
            "Nº de Isolados de Staphylococcus aureus",
            "Nº de Isolados de Enterococcus spp.",
            "% de A. baumannii resistente a Carbapenêmicos",
            "% de S. aureus resistente à Oxacilina (MRSA)",
            "% de Enterococcus resistente à Vancomicina (VRE)",
        ],
        "Valor": [
            total_pac_positivos,
            total_pac_negativos,
            total_amostras,
            total_isolados,
            isolados_acineto,
            isolados_klebsiella,
            isolados_pseudo,
            isolados_staph,
            isolados_entero,
            f"{taxa_acineto_carba:.1f}%",
            f"{taxa_mrsa:.1f}%",
            f"{taxa_vre:.1f}%",
        ],
    }
    return pd.DataFrame(data)


def gerar_aba_perfil_sensibilidade(df: pd.DataFrame) -> pd.DataFrame:
    """Gera a aba de Perfil de Sensibilidade (Antibiograma)."""
    df_analise = df[df["microorganismo"] != "N/A"].copy()

    pivot = df_analise.pivot_table(
        index=["microorganismo", "antibiotico_testado"],
        columns="resultado_rsi",
        values="prontuario",
        aggfunc="count",
        fill_value=0,
    )

    for col in ["S", "R", "I"]:
        if col not in pivot.columns:
            pivot[col] = 0

    pivot["Nº de Testes"] = pivot.sum(axis=1)

    pivot["% Sensível (S)"] = (pivot["S"] / pivot["Nº de Testes"] * 100).round(1)
    pivot["% Intermediário (I)"] = (pivot["I"] / pivot["Nº de Testes"] * 100).round(1)
    pivot["% Resistente (R)"] = (pivot["R"] / pivot["Nº de Testes"] * 100).round(1)

    return pivot.reset_index()[
        [
            "microorganismo",
            "antibiotico_testado",
            "Nº de Testes",
            "% Sensível (S)",
            "% Intermediário (I)",
            "% Resistente (R)",
        ]
    ]


# **AQUI ESTÁ A MUDANÇA PRINCIPAL**: Novo formato para a aba de Unidade
def gerar_aba_distribuicao_unidade(df: pd.DataFrame) -> pd.DataFrame:
    """
    Gera a aba de Distribuição de Microrganismos por Unidade/Setor em formato LONGO.
    """
    df_analise = df[df["microorganismo"] != "N/A"].copy()

    distribuicao = (
        df_analise.groupby(["unidade_solicitante", "microorganismo"])["prontuario"]
        .nunique()
        .reset_index()
    )

    distribuicao = distribuicao.rename(columns={"prontuario": "Nº de Pacientes Únicos"})

    distribuicao = distribuicao.sort_values(
        by=["unidade_solicitante", "Nº de Pacientes Únicos"], ascending=[True, False]
    )

    return distribuicao


# **AQUI ESTÁ A MUDANÇA PRINCIPAL**: Novo formato para a aba de Amostra
def gerar_aba_distribuicao_amostra(df: pd.DataFrame) -> pd.DataFrame:
    """
    Gera a aba de Distribuição de Microrganismos por Tipo de Amostra em formato LONGO.
    """
    df_analise = df[
        (df["microorganismo"] != "N/A") & (df["tipo_amostra"] != "N/A")
    ].copy()

    if df_analise.empty:
        return pd.DataFrame(
            {
                "Tipo de Amostra": ["Nenhum dado de amostra encontrado para análise."],
                "Microrganismo": [""],
                "Nº de Pacientes Únicos": [0],
            }
        )

    distribuicao = (
        df_analise.groupby(["tipo_amostra", "microorganismo"])["prontuario"]
        .nunique()
        .reset_index()
    )

    distribuicao = distribuicao.rename(columns={"prontuario": "Nº de Pacientes Únicos"})

    distribuicao = distribuicao.sort_values(
        by=["tipo_amostra", "Nº de Pacientes Únicos"], ascending=[True, False]
    )

    return distribuicao


def gerar_aba_base_completa(df: pd.DataFrame) -> pd.DataFrame:
    """Gera a aba com os dados detalhados e selecionados."""
    colunas_selecionadas = [
        "data_liberacao",
        "nome_paciente",
        "prontuario",
        "unidade_solicitante",
        "tipo_amostra",
        "microorganismo",
        "antibiotico_testado",
        "resultado_rsi",
        "valor_mic",
        "origem_atendimento_simplificada",
        "tipo_alta",
        "perfil_resistencia",
    ]
    colunas_existentes = [col for col in colunas_selecionadas if col in df.columns]
    return df[colunas_existentes]

In [22]:
# ==============================================================================
# CÉLULA 6: FUNÇÃO PARA SALVAR O ARQUIVO EXCEL
# ==============================================================================
def salvar_relatorio_excel(caminho_saida: Path, abas: dict):
    """
    Salva os DataFrames em um arquivo Excel, com cada um em uma aba.
    Aplica formatação automática de largura de coluna.

    Args:
        caminho_saida (Path): O caminho completo do arquivo .xlsx a ser salvo.
        abas (dict): Um dicionário onde as chaves são os nomes das abas e os
                     valores são os DataFrames a serem salvos.
    """
    try:
        with pd.ExcelWriter(caminho_saida, engine="openpyxl") as writer:
            for nome_aba, df_aba in abas.items():
                df_aba.to_excel(writer, sheet_name=nome_aba, index=False)

                # Bloco para auto-ajuste da largura das colunas
                ws = writer.sheets[nome_aba]
                for column in ws.columns:
                    max_length = 0
                    column_letter = column[0].column_letter
                    for cell in column:
                        try:
                            if len(str(cell.value)) > max_length:
                                max_length = len(cell.value)
                        except:
                            pass
                    adjusted_width = max_length + 2
                    ws.column_dimensions[column_letter].width = adjusted_width

        print(f"✅ Relatório salvo com sucesso em: {caminho_saida}")

    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo Excel: {e}")

In [ ]:
# ==============================================================================
# CÉLULA 7: FUNÇÃO ORQUESTRADORA PRINCIPAL (VERSÃO CORRIGIDA E FINAL)
# ==============================================================================
def gerar_relatorio_cciras_mensal(
    mes_ano_base: str, pasta_dados: Path, pasta_saida: Path
):
    """
    Executa o pipeline completo para gerar o relatório da CCIRAS para um mês.
    """
    # 1. Consolidar os dados brutos usando a nova lógica de merge
    df_raw = consolidar_dados_mensais(mes_ano_base, pasta_dados)
    if df_raw is None:
        return

    # 2. Ler arquivos de contagem
    try:
        df_pos = pd.read_csv(pasta_dados / "Contagem pacientes amostra positiva.csv")
        df_neg = pd.read_csv(pasta_dados / "Contagem pacientes amostras negativas.csv")
        contagens = {"positivos": len(df_pos), "negativos": len(df_neg)}
    except FileNotFoundError:
        print(
            "⚠️ Arquivos de contagem não encontrados. O Dashboard terá valores zerados."
        )
        contagens = {"positivos": 0, "negativos": 0}

    # 3. Limpar e preparar os dados
    df_clean = limpar_e_preparar_dados(df_raw)

    # 4. Gerar cada aba analítica
    abas_do_relatorio = {
        "Dashboard Indicadores": gerar_aba_dashboard(df_clean, contagens),
        "Perfil de Sensibilidade": gerar_aba_perfil_sensibilidade(df_clean),
        "Distribuição por Unidade": gerar_aba_distribuicao_unidade(df_clean),
        "Distribuição por Amostra": gerar_aba_distribuicao_amostra(df_clean),
        "Base de Dados Completa": gerar_aba_base_completa(df_clean),
    }

    # 5. Salvar o arquivo Excel
    nome_arquivo_saida = f"Relatorio_CCIRAS_HC-UFPE_{mes_ano_base}.xlsx"
    caminho_completo_saida = pasta_saida / nome_arquivo_saida
    salvar_relatorio_excel(caminho_completo_saida, abas_do_relatorio)

In [24]:
# ==============================================================================
# CÉLULA 8: EXECUÇÃO DO PROCESSO (VERSÃO CORRIGIDA)
# ==============================================================================
def processar_ano_completo(ano_2d: str, pasta_dados: Path, pasta_saida: Path):
    """
    Encontra todos os arquivos de um ano e executa o gerador de relatórios para cada um.
    """
    print(f"\n\n{'='*20} INICIANDO PROCESSAMENTO PARA O ANO 20{ano_2d} {'='*20}")

    # **AQUI ESTÁ A MUDANÇA PRINCIPAL**: Padrão para encontrar arquivos como 'vigiram-jan25.csv'
    padrao = re.compile(rf"vigiram-([a-z]{{3}}{ano_2d})\.csv", re.IGNORECASE)

    arquivos_encontrados = [padrao.match(f) for f in os.listdir(pasta_dados)]
    bases_de_mes = sorted(list(set(m.group(1) for m in arquivos_encontrados if m)))

    if not bases_de_mes:
        print(
            f"Nenhuma base de dados 'vigiram-*.csv' encontrada para o ano 20{ano_2d} na pasta {pasta_dados}"
        )
        return

    print(f"Bases 'vigiram' encontradas para 20{ano_2d}: {', '.join(bases_de_mes)}")

    for mes_ano_base in bases_de_mes:
        gerar_relatorio_cciras_mensal(mes_ano_base, pasta_dados, pasta_saida)
        print("-" * 70)

    print(f"🏁 Processamento para o ano 20{ano_2d} finalizado. 🏁")


# --- PONTO DE PARTIDA: DEFINA O ANO A SER PROCESSADO AQUI ---
ANO_A_PROCESSAR = "25"  # Ex: "24" para 2024, "25" para 2025
processar_ano_completo(ANO_A_PROCESSAR, DATA_DIR, OUT_DIR)



==================== INICIANDO PROCESSAMENTO PARA O ANO 2025 ====================
Bases 'vigiram' encontradas para 2025: abr25, fev25, jan25, jun25, mai25, mar25
🔄 Processando base: abr25...
✅ Dados de 'abr25' consolidados. Fonte de amostra ('Material Análise' do ITS) vinculada.
✅ Relatório salvo com sucesso em: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\Relatorio_CCIRAS_HC-UFPE_abr25.xlsx
----------------------------------------------------------------------
🔄 Processando base: fev25...
✅ Dados de 'fev25' consolidados. Fonte de amostra ('Material Análise' do ITS) vinculada.
✅ Relatório salvo com sucesso em: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\outputs\Relatorio_CCIRAS_HC-UFPE_fev25.xlsx
----------------------------------------------------------------------
🔄 Processando base: jan25...
✅ Dados de 'jan25' consolidados. Fonte de amostra ('Material Análise' do ITS) vinculada.
✅ Relatório salvo com sucesso em: C:\Users\marcelo.petry\Documents\CCIRAS\vigiram\output